In [67]:
import numpy as np
# this function simulate 1 batch of K number of X_n, given fixed Lambda, P, Alpha
# @ param N: is the number of total population
# @ param total_time: the time that we want to simulate the process (6 month)
# @ param x_0: initial infected patients (greater than 0)
def simulation_infected(la, p, al, N, total_time, x_0, K):
    # initialize time and infected patient
    t_0 = 0
    # bind variables to initial conditions
    x_n = x_0
    assert(x_n!=0)
    t_n = t_0
    # initiate X_n process and corresponding T_n process
    X_n = [x_0]
    T_n = [t_0]
    # converge
    k = 0
    # loop
    # break condition
    # @ condition1: when the time exceeds the total time
    # @ condition2: when the infected patients go to zero
    # @ condition3: when the whole populations are infected! :(
    while (t_n<total_time and
           x_n != 0 and
           x_n < N and
           k < K
          ):
        i = x_n
        # q i _ i+1
        q_forward_i = la*p*2*x_n*(N-i)/(N*(N-1))
        # q i _ i-1
        q_backward_i = al*i
        # waiting time rate v_i = (q i _ i+1) + (q i _ i-1)
        v_i = q_forward_i + q_backward_i
        t_i = np.random.exponential(v_i)
        # jumping probability to STATE i+1 is (q i _ i+1)/v_i
        jump = np.random.binomial(n=1,p=(q_forward_i/v_i))
        if (jump ==1):
            x_n += 1
        else:
            x_n -= 1
        # increase time    
        t_n = t_n+t_i
        # append Process
        X_n.append(x_n)
        T_n.append(t_n)
        # increment
        k +=1         
    return X_n,T_n

In [68]:
# @ C is the hospital capacity
# @ Beta is the target tracking probability
# @ K is the batch size of each estimate
def Y_n(X_n,C,Beta):
    overCapacityCount = 0
    K = len(X_n)
    for i in range(K):
        if X_n[i] > C:
            overCapacityCount+=1
    Y_n = (overCapacityCount/K) - Beta
    return Y_n
    
def update_lambda(_lambda, Y_n, stepsize):
    _lambda -= float(stepsize) * Y_n
    return _lambda

def update_alpha(_alpha, Y_n, stepsize):
    _alpha -= float(stepsize) * Y_n
    return _alpha

# @ x_n initial infected patient
# set initial lambda
# set inital alpha
# @ C is the hospital capacity 
# @ Beta is the target tracking probability 
# @ K is the batch size of each estimate
# these are all set arbitary atm
x_n, _lambda,_alpha,p, K, H, Beta, N, totalTime, stepsize = (19999,
                                                0.1,
                                                1e-8,
                                                0.1,
                                                5,
                                                2e4,
                                                0.05,
                                                4e6,
                                                30000,
                                                0.01)



trajectory_lambda = []
trajectory_alpha = []
trajectory_X_n = []
trajectory_total_T = []
# use iteration of 5000 as end of all sequential update
iteration = 0
while iteration < 50000:
    # update alpha
    X_n,T_n = simulation_infected(_lambda,p,_alpha,N,totalTime,x_n,K)
#     print(_alpha)
    print("###############")
    print(X_n)
    _alpha = max(0.001,update_alpha(_alpha, Y_n(X_n,C,Beta),stepsize))
    trajectory_alpha.append(_alpha)
    trajectory_X_n.extend(X_n)
    trajectory_total_T.extend(T_n)
    
    # to keep the time consistent, slice the tail of last number of infected
    # as the new initial
    x_n = X_n[-1]
    
    # update lambda
    X_n,T_n = simulation_infected(_lambda, p, _alpha, N,totalTime, x_n, K)
    print("***************")
    print(X_n)
    _lambda = max(0.001, update_lambda(_lambda,Y_n(X_n,C,Beta),stepsize))
    
    trajectory_lambda.append(_lambda)
    trajectory_X_n.extend(X_n)
    trajectory_total_T.extend(T_n)
    
    x_n = X_n[-1]
    
    iteration+=1
    

###############
[19999, 19998, 19997, 19996, 19995, 19994]
***************
[19994, 19993, 19992, 19991, 19990, 19989]
###############
[19989, 19988, 19987, 19986, 19985, 19984]
***************
[19984, 19983, 19982, 19981, 19980, 19979]
###############
[19979, 19978, 19977, 19976, 19975, 19974]
***************
[19974, 19973, 19972, 19971, 19970, 19969]
###############
[19969, 19968, 19967, 19966, 19965, 19964]
***************
[19964, 19963, 19962, 19961, 19960, 19959]
###############
[19959, 19958, 19957, 19956, 19955, 19954]
***************
[19954, 19953, 19952, 19951, 19950, 19949]
###############
[19949, 19948, 19947, 19946, 19945, 19944]
***************
[19944, 19943, 19942, 19941, 19940, 19939]
###############
[19939, 19938, 19937, 19936, 19935, 19934]
***************
[19934, 19933, 19932, 19931, 19930, 19929]
###############
[19929, 19928, 19927, 19926, 19925, 19924]
***************
[19924, 19923, 19922, 19921, 19920, 19919]
###############
[19919, 19918, 19917, 19916, 19915, 1991

***************
[16614, 16613, 16612, 16611, 16610, 16609]
###############
[16609, 16608, 16607, 16606, 16605, 16604]
***************
[16604, 16603, 16602, 16601, 16600, 16599]
###############
[16599, 16598, 16597, 16596, 16595, 16594]
***************
[16594, 16593, 16592, 16591, 16590, 16589]
###############
[16589, 16588, 16587, 16586, 16585, 16584]
***************
[16584, 16583, 16582, 16581, 16580, 16579]
###############
[16579, 16578, 16577, 16576, 16575, 16574]
***************
[16574, 16573, 16572, 16571, 16570, 16569]
###############
[16569, 16568, 16567, 16566, 16565, 16564]
***************
[16564, 16563, 16562, 16561, 16560, 16559]
###############
[16559, 16558, 16557, 16556, 16555, 16554]
***************
[16554, 16553, 16552, 16551, 16550, 16549]
###############
[16549, 16548, 16547, 16546, 16545, 16544]
***************
[16544, 16543, 16542, 16541, 16540, 16539]
###############
[16539, 16538, 16537, 16536, 16535, 16534]
***************
[16534, 16533, 16532, 16531, 16530, 1652

***************
[14114, 14113, 14112, 14111, 14110, 14109]
###############
[14109, 14108, 14107, 14106, 14105, 14104]
***************
[14104, 14103, 14102, 14101, 14100, 14099]
###############
[14099, 14098, 14097, 14096, 14095, 14094]
***************
[14094, 14093, 14092, 14091, 14090, 14089]
###############
[14089, 14088, 14087, 14086, 14085, 14084]
***************
[14084, 14083, 14082, 14081, 14080, 14079]
###############
[14079, 14078, 14077, 14076, 14075, 14074]
***************
[14074, 14073, 14072, 14071, 14070, 14069]
###############
[14069, 14068, 14067, 14066, 14065, 14064]
***************
[14064, 14063, 14062, 14061, 14060, 14059]
###############
[14059, 14058, 14057, 14056, 14055, 14054]
***************
[14054, 14053, 14052, 14051, 14050, 14049]
###############
[14049, 14048, 14047, 14046, 14045, 14044]
***************
[14044, 14043, 14042, 14041, 14040, 14039]
###############
[14039, 14038, 14037, 14036, 14035, 14034]
***************
[14034, 14033, 14032, 14031, 14030, 1402

###############
[11759, 11758, 11757, 11756, 11755, 11754]
***************
[11754, 11753, 11752, 11751, 11750, 11749]
###############
[11749, 11748, 11747, 11746, 11745, 11744]
***************
[11744, 11743, 11742, 11741, 11740, 11739]
###############
[11739, 11738, 11737, 11736, 11735, 11734]
***************
[11734, 11733, 11732, 11731, 11730, 11729]
###############
[11729, 11728, 11727, 11726, 11725, 11724]
***************
[11724, 11723, 11722, 11721, 11720, 11719]
###############
[11719, 11718, 11717, 11716, 11715, 11714]
***************
[11714, 11713, 11712, 11711, 11710, 11709]
###############
[11709, 11708, 11707, 11706, 11705, 11704]
***************
[11704, 11703, 11702, 11701, 11700, 11699]
###############
[11699, 11698, 11697, 11696, 11695, 11694]
***************
[11694, 11693, 11692, 11691, 11690, 11689]
###############
[11689, 11688, 11687, 11686, 11685, 11684]
***************
[11684, 11683, 11682, 11681, 11680, 11679]
###############
[11679, 11678, 11677, 11676, 11675, 1167

[10369, 10368, 10367, 10366, 10365, 10364]
***************
[10364, 10363, 10362, 10361, 10360, 10359]
###############
[10359, 10358, 10357, 10356, 10355, 10354]
***************
[10354, 10353, 10352, 10351, 10350, 10349]
###############
[10349, 10348, 10347, 10346, 10345, 10344]
***************
[10344, 10343, 10342, 10341, 10340, 10339]
###############
[10339, 10338, 10337, 10336, 10335, 10334]
***************
[10334, 10333, 10332, 10331, 10330, 10329]
###############
[10329, 10328, 10327, 10326, 10325, 10324]
***************
[10324, 10323, 10322, 10321, 10320, 10319]
###############
[10319, 10318, 10317, 10316, 10315, 10314]
***************
[10314, 10313, 10312, 10311, 10310, 10309]
###############
[10309, 10308, 10307, 10306, 10305, 10304]
***************
[10304, 10303, 10302, 10301, 10300, 10299]
###############
[10299, 10298, 10297, 10296, 10295, 10294]
***************
[10294, 10293, 10292, 10291, 10290, 10289]
###############
[10289, 10288, 10287, 10286, 10285, 10284]
*************

...

###############
[5579, 5578, 5577, 5576, 5575, 5574]
***************
[5574, 5573, 5572, 5571, 5570, 5569]
###############
[5569, 5568, 5567, 5566, 5565, 5564]
***************
[5564, 5563, 5562, 5561, 5560, 5559]
###############
[5559, 5558, 5557, 5556, 5555, 5554]
***************
[5554, 5553, 5552, 5551, 5550, 5549]
###############
[5549, 5548, 5547, 5546, 5545, 5544]
***************
[5544, 5543, 5542, 5541, 5540, 5539]
###############
[5539, 5538, 5537, 5536, 5535, 5534]
***************
[5534, 5533, 5532, 5531, 5530, 5529]
###############
[5529, 5528, 5527, 5526, 5525, 5524]
***************
[5524, 5523, 5522, 5521, 5520, 5519]
###############
[5519, 5518, 5517, 5516, 5515, 5514]
***************
[5514, 5513, 5512, 5511, 5510, 5509]
###############
[5509, 5508, 5507, 5506, 5505, 5504]
***************
[5504, 5503, 5502, 5501, 5500, 5499]
###############
[5499, 5498, 5497, 5496, 5495, 5494]
***************
[5494, 5493, 5492, 5491, 5490, 5489]
###############
[5489, 5488, 5487, 5486, 5485,

[4124, 4123, 4122, 4121, 4120, 4119]
###############
[4119, 4118, 4117, 4116, 4115, 4114]
***************
[4114, 4113, 4112, 4111, 4110, 4109]
###############
[4109, 4108, 4107, 4106, 4105, 4104]
***************
[4104, 4103, 4102, 4101, 4100, 4099]
###############
[4099, 4098, 4097, 4096, 4095, 4094]
***************
[4094, 4093, 4092, 4091, 4090, 4089]
###############
[4089, 4088, 4087, 4086, 4085, 4084]
***************
[4084, 4083, 4082, 4081, 4080, 4079]
###############
[4079, 4078, 4077, 4076, 4075, 4074]
***************
[4074, 4073, 4072, 4071, 4070, 4069]
###############
[4069, 4068, 4067, 4066, 4065, 4064]
***************
[4064, 4063, 4062, 4061, 4060, 4059]
###############
[4059, 4058, 4057, 4056, 4055, 4054]
***************
[4054, 4053, 4052, 4051, 4050, 4049]
###############
[4049, 4048, 4047, 4046, 4045, 4044]
***************
[4044, 4043, 4042, 4041, 4040, 4039]
###############
[4039, 4038, 4037, 4036, 4035, 4034]
***************
[4034, 4033, 4032, 4031, 4030, 4029]
#########

[2179, 2178, 2177, 2176, 2175, 2174]
***************
[2174, 2173, 2172, 2171, 2170, 2169]
###############
[2169, 2168, 2167, 2166, 2165, 2164]
***************
[2164, 2163, 2162, 2161, 2160, 2159]
###############
[2159, 2158, 2157, 2156, 2155, 2154]
***************
[2154, 2153, 2152, 2151, 2150, 2149]
###############
[2149, 2148, 2147, 2146, 2145, 2144]
***************
[2144, 2143, 2142, 2141, 2140, 2139]
###############
[2139, 2138, 2137, 2136, 2135, 2134]
***************
[2134, 2133, 2132, 2131, 2130, 2129]
###############
[2129, 2128, 2127, 2126, 2125, 2124]
***************
[2124, 2123, 2122, 2121, 2120, 2119]
###############
[2119, 2118, 2117, 2116, 2115, 2114]
***************
[2114, 2113, 2112, 2111, 2110, 2109]
###############
[2109, 2108, 2107, 2106, 2105, 2104]
***************
[2104, 2103, 2102, 2101, 2100, 2099]
###############
[2099, 2098, 2097, 2096, 2095, 2094]
***************
[2094, 2093, 2092, 2091, 2090, 2089]
###############
[2089, 2088, 2087, 2086, 2085, 2084]
*********

***************
[374, 373, 372, 371, 370, 369]
###############
[369, 368, 367, 366, 365, 364]
***************
[364, 363, 362, 361, 360, 359]
###############
[359, 358, 357, 356, 355, 354]
***************
[354, 353, 352, 351, 350, 349]
###############
[349, 348, 347, 346, 345, 344]
***************
[344, 343, 342, 341, 340, 339]
###############
[339, 338, 337, 336, 335, 334]
***************
[334, 333, 332, 331, 330, 329]
###############
[329, 328, 327, 326, 325, 324]
***************
[324, 323, 322, 321, 320, 319]
###############
[319, 318, 317, 316, 315, 314]
***************
[314, 313, 312, 311, 310, 309]
###############
[309, 308, 307, 306, 305, 304]
***************
[304, 303, 302, 301, 300, 299]
###############
[299, 298, 297, 296, 295, 294]
***************
[294, 293, 292, 291, 290, 289]
###############
[289, 288, 287, 286, 285, 284]
***************
[284, 283, 282, 281, 280, 279]
###############
[279, 278, 277, 276, 275, 274]
***************
[274, 273, 272, 271, 270, 269]
#############

AssertionError: 